## Validate the audio files

In [ ]:
AUDIO_DIR = "PATH_TO_AUDIO_FILES"

In [ ]:
import librosa

MIN_LENGTH = 1024  #  Minimum samples
EXPECTED_SAMPLE_RATE = 22050  # Expected sampling-rate

def validate_audio_files(audio_dir, min_length, expected_sample_rate):
    """
    Validates all audio files in a directory.
    Checks file size, length and sampling rate.

    :param audio_dir: Pfad zum Verzeichnis mit Audiodateien
    :param min_length: Mindestanzahl an Samples (FFT-Größe)
    :param expected_sample_rate: Erwartete Sampling-Rate
    """
    invalid_files = []

    for root, _, files in os.walk(audio_dir):
        for file in files:
            file_path = os.path.join(root, file)

            # Skip files that are not audio
            if not file.endswith((".wav", ".flac", ".mp3")):
                continue

            try:
                # Check file size
                if os.path.getsize(file_path) == 0:
                    print(f"Empty file: {file_path}")
                    invalid_files.append(file_path)
                    continue

                # Load audio file
                audio, sr = librosa.load(file_path, sr=None)

                # Check Sampling-Rate
                if sr != expected_sample_rate:
                    print(f"Inconsistent Sampling-Rate ({sr} instead of {expected_sample_rate}): {file_path}")
                    invalid_files.append(file_path)
                    continue

                # Prüfe Audiodauer
                if len(audio) < min_length:
                    print(f"File with ({len(audio)} Samples too short): {file_path}")
                    invalid_files.append(file_path)
                    continue

            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                invalid_files.append(file_path)

    print("\nValidation completed.")
    if invalid_files:
        print(f"{len(invalid_files)} invalid files found:")
        for f in invalid_files:
            print(f" - {f}")
    else:
        print("All files are valid!")

if __name__ == "__main__":
    validate_audio_files(AUDIO_DIR, MIN_LENGTH, EXPECTED_SAMPLE_RATE)

## Generate

In [ ]:
import os
from TTS.utils.synthesizer import Synthesizer
from TTS.config import load_config
from TTS.utils.audio import AudioProcessor

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

def main():
    model_path = "PATH_TO_MODEL" 
    config_path = "PATH_TO_MODEL_CONFIG"

    # Laden der Konfiguration
    config = load_config(config_path)

    # Initialisierung des Synthesizers
    synthesizer = Synthesizer(
        tts_checkpoint=model_path,
        tts_config_path=config_path,  
        tts_speakers_file=None,
        use_cuda=True  # Set to true if cuda is used
    )

    # Example sentences
    sentences = [
        "Why are you doing that?",
        "This process can be hard to understand sometimes.",
        "Sentence after sentence will be generated!"
    ]

    # Generate audio and save
    output_folder = "synthesized_audio"
    os.makedirs(output_folder, exist_ok=True)

    for i, sentence in enumerate(sentences):
        print(f"Generating sentence {i+1}/{len(sentences)}: {sentence}")
        wav = synthesizer.tts(sentence)
        synthesizer.save_wav(wav, os.path.join(output_folder, f"sentence_{i+1}.wav"))

if __name__ == "__main__":
    main()